**Import required libraries and scripts**

In [ ]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *
from scripts.postprocessing import *
from scripts.protein_preparation import *

In [ ]:
CWD = os.getcwd()
software = Path(CWD+'/software')
receptor = Path(CWD+'/testing_single_docking/protein.pdb')
ref_file = Path(CWD+'/testing_single_docking/ref.sdf')
docking_library = Path(CWD+'/testing_single_docking/library.sdf')
docking_programs = ['PLANTS']
clustering_metrics = list(CLUSTERING_METRICS.keys())+['bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS', 'bestpose_QVINAW', 'bestpose_QVINA2']
rescoring_functions = list(RESCORING_FUNCTIONS.keys())
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
conformers = 'GypsumDL'
protonation = 'GypsumDL'
ncpus = 5
pocket = 'reference'
prepare_protein = True
open('log.txt', 'w').close()

In [ ]:
if prepare_protein == True:
    #Prepare the protein
    prepared_receptor = prepare_protein_protoss(receptor)
else:
    prepared_receptor = receptor

#Create a temporary folder for all further calculations
w_dir = prepared_receptor.parent / prepared_receptor.stem
print('The working directory has been set to:', w_dir)
(w_dir).mkdir(exist_ok=True)

if pocket == 'reference':
    pocket_definition = get_pocket(ref_file, prepared_receptor, 8)
    print(pocket_definition)
if pocket == 'RoG':
    pocket_definition = get_pocket_RoG(ref_file, prepared_receptor)
    print(pocket_definition)
elif pocket == 'dogsitescorer':
    pocket_definition = binding_site_coordinates_dogsitescorer(prepared_receptor, w_dir, method='volume')
    print(pocket_definition)

In [ ]:
if (w_dir / 'final_library.sdf').is_file() == False:
    prepare_library(docking_library, w_dir, id_column, conformers, protonation, software, ncpus)

In [ ]:
t1 = time.perf_counter()
docking(w_dir, prepared_receptor, pocket_definition, software, docking_programs, exhaustiveness, n_poses, 5, 'concurrent_process')
t2 = time.perf_counter()
print(f'Took : {t2-t1}')

In [ ]:
t1 = time.perf_counter()
docking(w_dir, prepared_receptor, pocket_definition, software, docking_programs, exhaustiveness, n_poses, 5, 'concurrent_thread')
t2 = time.perf_counter()
print(f'Took : {t2-t1}')

In [ ]:
t1 = time.perf_counter()
docking(w_dir, prepared_receptor, pocket_definition, software, docking_programs, exhaustiveness, n_poses, 5, 'joblib')
t2 = time.perf_counter()
print(f'Took : {t2-t1}')

In [ ]:
t1 = time.perf_counter()
docking(w_dir, prepared_receptor, pocket_definition, software, docking_programs, exhaustiveness, n_poses, 5, 'pebble_process')
t2 = time.perf_counter()
print(f'Took : {t2-t1}')

In [ ]:
t1 = time.perf_counter()
docking(w_dir, prepared_receptor, pocket_definition, software, docking_programs, exhaustiveness, n_poses, 5, 'pebble_thread')
t2 = time.perf_counter()
print(f'Took : {t2-t1}')

In [ ]:
t1 = time.perf_counter()
docking(w_dir, prepared_receptor, pocket_definition, software, docking_programs, exhaustiveness, n_poses, 5, 'ray')
t2 = time.perf_counter()
print(f'Took : {t2-t1}')

In [ ]:
concat_all_poses(w_dir, docking_programs, prepared_receptor, ncpus)

In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(str(w_dir / 'allposes.sdf'), idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
print(f'Loaded {len(all_poses)} poses.')
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')


In [ ]:
for metric in clustering_metrics:
    if os.path.isfile(w_dir / f'clustering/{metric}_clustered.sdf') == False:
        cluster_pebble(metric, 'KMedoids', w_dir, prepared_receptor, pocket_definition, software, all_poses, ncpus)

In [ ]:
for metric in clustering_metrics:
        rescore_poses(w_dir, prepared_receptor, pocket_definition, software, w_dir / 'clustering' / f'{metric}_clustered.sdf', rescoring_functions , ncpus)

In [ ]:
apply_consensus_methods(w_dir, 'bestpose', 'ECR_best', rescoring_functions, 'min_max')